# Pathfinding Challenge

In [ ]:
import numpy as np
import pyvista as pv
import shapely as sh
import matplotlib.pyplot as plt

from airfoil.util.array_helpers import split_indexable
from airfoil.util.shapely_helpers import (
    plot_shapely,
    plot_shapely_directional,

)
from airfoil.util.linestring_helpers import (
    deflection_angle
)

In [ ]:
s0 = sh.Point(0,0).buffer(60)
s1 = sh.Point(0,0).buffer(60+30)
s2 = sh.box(-100,-100,100,0)
s3 = sh.box(-15,50, 15,250)
spork:sh.Polygon = sh.difference(sh.union(s3,sh.difference(s1,s2)),s0)

fig, (axa, axb) = plt.subplots(1,2,figsize=(10,5))
plot_shapely([s0,s1,s2,s3], ax=axa)
plot_shapely_directional([spork],legend=["spork"],ax=axb)

In [ ]:
segments = split_indexable(
    spork.exterior.coords, 
    np.where(deflection_angle(spork.exterior.coords)>np.deg2rad(30))[0]+1
)
plot_shapely_directional([sh.MultiLineString(segments)])

In [ ]:
sh.MultiPolygon([s for s in sh.delaunay_triangles(spork).geoms if spork.contains(s.centroid)])

In [ ]:
spork2 = sh.affinity.translate(
    sh.affinity.rotate(spork,180,origin=(0,0)),
    70,20
)
plot_shapely_directional([spork, spork2])